# RePlay bandit models comparison

We will show the main RePlay functionality and compare performance of RePlay models on well-known MovieLens dataset. For simplicity we consider here only the various bandit algorithms, both context-free and context-aware. The list of considered strategies for comparison:

Context-free algorithms:
* Most popular;
* Vanilla UCB algorithm;
* Vanilla TS algorithm; (Beta-Binomial);
* KL-UCB [Smb et al];

Contextual bandits algorithms:
* Lin-UCB [Smb et al];
* Linear TS [Smb et al] (Thompson sampling with linear feature vectors);


### Dataset
We will compare RePlay models on __MovieLens 1m__. 

### Dataset preprocessing: 
Ratings greater than or equal to 3 are considered as positive interactions.

### Data split
Dataset is split by date so that 20% of the last interactions as are placed in the test part. Cold items and users are dropped.

### Predict:
We will predict top-10 most relevant films for each user.

### Metrics
Quality metrics used:__ndcg@k, hitrate@k, map@k, mrr@k__ for k = 1, 5, 10
Additional metrics used: __coverage@k__ and __surprisal@k__.

In [1]:
# ! pip install rs-datasets

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%config Completer.use_jedi = False

In [4]:
import warnings
from optuna.exceptions import ExperimentalWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ExperimentalWarning)

`State` object allows passing existing Spark session or create a new one, which will be used by the all RePlay modules.

To create session with custom parameters ``spark.driver.memory`` and ``spark.sql.shuffle.partitions`` use function `get_spark_session` from `session_handler` module.

In [5]:
import logging
import time

from pyspark.sql import functions as sf, types as st
from pyspark.sql.types import IntegerType

from replay.data import Dataset, FeatureHint, FeatureInfo, FeatureSchema, FeatureType
from replay.experimental.preprocessing.data_preparator import Indexer, DataPreparator
from replay.metrics import Experiment
from replay.metrics import Coverage, HitRate, MRR, MAP, NDCG, Surprisal
from replay.models import (
    PopRec, 
    RandomRec,
    UCB,
    Wilson, 
    ThompsonSampling,
    LinUCB, # added LinUCB 
)

from replay.models.base_rec import HybridRecommender
from replay.utils.session_handler import State
from replay.splitters import TimeSplitter
from replay.utils.spark_utils import convert2spark, get_log_info
from rs_datasets import MovieLens

import pandas as pd
import numpy as np

In [ ]:
spark = State().session
spark

In [7]:
spark.sparkContext.setLogLevel('ERROR')

In [8]:
logger = logging.getLogger("replay")

In [10]:
K = 10
K_list_metrics = [1, 5, 10]
BUDGET = 5
BUDGET_NN = 2
SEED = 12345

## 0. Preprocessing <a name='data-preparator'></a>

### 0.1 Data loading

In [11]:
data = MovieLens("1m")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [12]:
data.ratings

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


#### log preprocessing

- converting to spark dataframe
- renaming columns
- checking for nulls
- converting timestamp to Timestamp format

In [13]:
preparator = DataPreparator()

In [14]:
%%time
log = preparator.transform(columns_mapping={'user_id': 'user_id',
                                      'item_id': 'item_id',
                                      'relevance': 'rating',
                                      'timestamp': 'timestamp'
                                     }, 
                           data=data.ratings)

03-Oct-24 17:32:03, replay, INFO: Columns with ids of users or items are present in mapping. The dataframe will be treated as an interactions log.
/home/sudakovcom/Desktop/RePlayHDILab2024/.conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:471: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


CPU times: user 18.8 ms, sys: 6.84 ms, total: 25.7 ms
Wall time: 2.43 s


In [15]:
# will consider ratings >= 3 as positive feedback. A positive feedback is treated with relevance = 1
only_positives_log = log.filter(sf.col('relevance') >= 3).withColumn('relevance', sf.lit(1))
only_positives_log.count()

836478

<a id='indexing'></a>
### 0.2. Indexing

Convert given users' and items' identifiers (\_id) to integers starting at zero without gaps (\_idx) with Indexer class.

In [16]:
indexer = Indexer(user_col='user_id', item_col='item_id')

Take all available user and item ids from log and features and pass them to Indexer. The _ids_ could repeat, the indexes will be ordered by label frequencies, so the most frequent label gets index 0.

In [17]:
%%time
indexer.fit(users=log.select('user_id'),
           items=log.select('item_id'))

CPU times: user 12.6 ms, sys: 4.69 ms, total: 17.3 ms
Wall time: 1.1 s


In [18]:
%%time
log_replay = indexer.transform(df=only_positives_log)
log_replay.show(2)
log_replay.count()

+--------+--------+---------+-------------------+
|user_idx|item_idx|relevance|          timestamp|
+--------+--------+---------+-------------------+
|    4131|      43|        1|2001-01-01 01:12:40|
|    4131|     585|        1|2001-01-01 01:35:09|
+--------+--------+---------+-------------------+
only showing top 2 rows

CPU times: user 17 ms, sys: 3.24 ms, total: 20.3 ms
Wall time: 890 ms


836478

### 0.2. Data split

In [19]:
# train/test split 
train_spl = TimeSplitter(
    time_threshold=0.2,
    drop_cold_items=True,
    drop_cold_users=True,
    query_column="user_idx",
    item_column="item_idx",
)

train, test = train_spl.split(log_replay)
print('train info:\n', get_log_info(train))
print('test info:\n', get_log_info(test))

train info:
 total lines: 669181, total users: 5397, total items: 3569


test info:
 total lines: 86542, total users: 1139, total items: 3279


In [21]:
# train/test split for hyperparameters selection
opt_train, opt_val = train_spl.split(train)
opt_train.count(), opt_val.count()

(535343, 24241)

In [22]:
# negative feedback will be used for Wilson and UCB models
only_negatives_log = indexer.transform(df=log.filter(sf.col('relevance') < 3).withColumn('relevance', sf.lit(0.)))
test_start = test.agg(sf.min('timestamp')).collect()[0][0]

# train with both positive and negative feedback
pos_neg_train=(train
              .withColumn('relevance', sf.lit(1.))
              .union(only_negatives_log.filter(sf.col('timestamp') < test_start))
             )
pos_neg_train.cache()
pos_neg_train.count()

798993

In [23]:
pos_neg_train.show(20)

+--------+--------+---------+-------------------+
|user_idx|item_idx|relevance|          timestamp|
+--------+--------+---------+-------------------+
|     677|    1314|      1.0|2000-12-02 08:30:12|
|     677|    1282|      1.0|2000-12-02 08:53:52|
|     677|     731|      1.0|2000-12-02 08:41:26|
|     677|     234|      1.0|2000-12-02 08:23:47|
|     677|     190|      1.0|2000-12-02 08:50:33|
|     677|     133|      1.0|2000-12-02 08:27:28|
|     677|     546|      1.0|2000-12-02 08:32:02|
|     677|    2090|      1.0|2000-12-02 08:53:17|
|     677|     421|      1.0|2000-12-02 08:50:13|
|     677|     154|      1.0|2000-12-02 08:44:14|
|     677|      96|      1.0|2000-12-02 08:11:05|
|     677|     221|      1.0|2000-12-02 08:58:08|
|     677|     395|      1.0|2000-12-02 08:33:02|
|     677|      19|      1.0|2000-12-02 08:46:50|
|     677|      73|      1.0|2000-12-02 08:31:30|
|     677|     182|      1.0|2000-12-02 08:18:10|
|     677|     836|      1.0|2000-12-02 08:47:30|


# 2. Models training

In [24]:
def fit_predict_add_res(name, model, experiment, train, test, suffix=''):
    """
    Run fit_predict for the `model`, measure time on fit_predict and evaluate metrics
    """
    start_time=time.time()
    
    dataset = {'dataset': train}
    predict_params = {'k': K, 'queries': test.interactions.select('user_idx').distinct()}
    
    if isinstance(model, (Wilson, UCB, ThompsonSampling, LinUCB)):
        dataset['dataset'] = train_neg_dataset
    
    predict_params.update(dataset)

    model.fit(**dataset)
    fit_time = time.time() - start_time

    pred=model.predict(**predict_params)
    pred.show(100)
    pred.cache()
    predict_time = time.time() - start_time - fit_time

    experiment.add_result(name + suffix, pred)
    metric_time = time.time() - start_time - fit_time - predict_time
    experiment.results.loc[name + suffix, 'fit_time'] = fit_time
    experiment.results.loc[name + suffix, 'predict_time'] = predict_time
    experiment.results.loc[name + suffix, 'metric_time'] = metric_time
    experiment.results.loc[name + suffix, 'full_time'] = (fit_time + 
                                                          predict_time +
                                                          metric_time)
    pred.unpersist()
    print(experiment.results[['NDCG@{}'.format(K), 'MRR@{}'.format(K), 'Coverage@{}'.format(K), 'fit_time']].sort_values('NDCG@{}'.format(K), ascending=False))

In [25]:
def full_pipeline(models, experiment, train, test, suffix='', budget=BUDGET):
    """
    For each model:
        -  if required: run hyperparameters search, set best params and save param values to `experiment`
        - pass model to `fit_predict_add_res`        
    """
    
    for name, [model, params] in models.items():
        model.logger.info(msg='{} started'.format(name))

        if params != 'no_opt':
            model.logger.info(msg='{} optimization started'.format(name))
            best_params = model.optimize(opt_train_dataset, 
                                         opt_val_dataset, 
                                         param_borders=params, 
                                         k=K, 
                                         budget=budget)
            logger.info(msg='best params for {} are: {}'.format(name, best_params))
            model.set_params(**best_params)

        
        logger.info(msg='{} fit_predict started'.format(name))
        fit_predict_add_res(name, model, experiment, train, test, suffix)
        # here we call protected attribute to get all parameters set during model initialization
        experiment.results.loc[name + suffix, 'params'] = str(model._init_args)

### 2.1. Contextual bandit models

### 2.1.1 item features preprocessing

In [26]:
item_features_original = preparator.transform(columns_mapping={'item_id': 'item_id'}, 
                           data=data.items)
item_features = indexer.transform(df=item_features_original)
item_features.show(2)
#different item features

from pyspark.sql.functions import max,min
item_features.select(max(item_features.item_idx)).show()
item_features.select(min(item_features.item_idx)).show()
#just to check that the indexing is dense between 0 and 3882
item_features.count()

03-Oct-24 17:32:47, replay, INFO: Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.
/home/sudakovcom/Desktop/RePlayHDILab2024/.conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:471: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


+--------+----------------+--------------------+
|item_idx|           title|              genres|
+--------+----------------+--------------------+
|      29|Toy Story (1995)|Animation|Childre...|
|     393|  Jumanji (1995)|Adventure|Childre...|
+--------+----------------+--------------------+
only showing top 2 rows

+-------------+
|max(item_idx)|
+-------------+
|         3882|
+-------------+

+-------------+
|min(item_idx)|
+-------------+
|            0|
+-------------+



3883

In [27]:
year = item_features.withColumn('year', sf.substring(sf.col('title'), -5, 4).astype(st.IntegerType())).select('item_idx', 'year')
year.show(2)

+--------+----+
|item_idx|year|
+--------+----+
|      29|1995|
|     393|1995|
+--------+----+
only showing top 2 rows



In [28]:
genres = (
    item_features.select(
        "item_idx",
        sf.split("genres", "\|").alias("genres")
    )
)

In [29]:
genres_list = (
    genres.select(sf.explode("genres").alias("genre"))
    .distinct().filter('genre <> "(no genres listed)"')
    .toPandas()["genre"].tolist()
)

In [30]:
genres_list

['Mystery',
 'Action',
 'Documentary',
 "Children's",
 'Drama',
 'Adventure',
 'Film-Noir',
 'Crime',
 'Animation',
 'Fantasy',
 'Comedy',
 'Western',
 'Romance',
 'Thriller',
 'War',
 'Sci-Fi',
 'Musical',
 'Horror']

In [31]:
item_features = genres
for genre in genres_list:
    item_features = item_features.withColumn(
        genre,
        sf.array_contains(sf.col("genres"), genre).astype(IntegerType())
    )
item_features = item_features.drop("genres").cache()
item_features.count()
item_features = item_features.join(year, on='item_idx', how='inner')
item_features.cache()

DataFrame[item_idx: int, Mystery: int, Action: int, Documentary: int, Children's: int, Drama: int, Adventure: int, Film-Noir: int, Crime: int, Animation: int, Fantasy: int, Comedy: int, Western: int, Romance: int, Thriller: int, War: int, Sci-Fi: int, Musical: int, Horror: int, year: int]

In [32]:
item_features = spark.createDataFrame(pd.concat([item_features.drop("year").toPandas(), pd.get_dummies(item_features.toPandas().year)], axis = 1))

/home/sudakovcom/Desktop/RePlayHDILab2024/.conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [33]:
item_features = item_features.drop("year")
item_features.show()

+--------+-------+------+-----------+----------+-----+---------+---------+-----+---------+-------+------+-------+-------+--------+---+------+-------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|item_idx|Mystery|Action|Documentary|Children's|Drama|Adventure|Film-Noir|Crime|Animation|Fantasy|Comedy|Western|Romance|Thriller|War|Sci-Fi|Musical|Horror|1919|1920|1921|1922|1923|1925|1926|1927|1928|1929|1930|1931|1932|1933|1934|1935|1936|1937|1938|1939|1940|1941|1942|1943|1944|1945|1946|1947|1948|1949|1950|1951|1952|1953|1954|1955|1956|1957|1958|1959|1960|1961|1962|1963|1964|1965|1966|1967|1968|1969|1970|1971|1972|1973|1974|1975|19

### 2.1.2 User features preprocessing

In [34]:
data.users.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [35]:
#same preprocessing for users as was done in 2.4.1.
user_features_original = preparator.transform(columns_mapping={'user_id': 'user_id'}, 
                           data=data.users)
user_features = indexer.transform(df=user_features_original)
#switch for a while into pandas
user_features = user_features.toPandas()
user_features.head(2)

03-Oct-24 17:33:02, replay, INFO: Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.
/home/sudakovcom/Desktop/RePlayHDILab2024/.conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:471: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


,user_idx,gender,age,occupation,zip_code
0,4131,F,1,10,48067
1,2364,M,56,16,70072


In [36]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
print("max ocupation index: ", user_features['occupation'].max())
print("min ocupation index: ", user_features['occupation'].min())
count_diff_zips = user_features['zip_code'].unique().size
print("different zip codes: ", count_diff_zips) #ok, too much different zip codes, let us drop them for now
users_pd = user_features.drop(columns=['zip_code'])
users_pd.head()
#binarize age variable
bins = [0, 20, 30, 40, 50, 60, np.inf]
names = ['<20', '20-29', '30-39','40-49', '51-60', '60+']

users_pd['agegroup'] = pd.cut(users_pd['age'], bins, labels=names)
users_pd = users_pd.drop(["age"], axis = 1)
users_pd.head()

#binarize following https://github.com/kfoofw/bandit_simulations/tree/master
columnsToEncode = ["agegroup", "gender", "occupation"]
oheEncoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
myEncoder.fit(users_pd[columnsToEncode])

users_pd = pd.concat([users_pd.drop(columnsToEncode, 1),
                           pd.DataFrame(myEncoder.transform(users_pd[columnsToEncode]), 
                                        columns = myEncoder.get_feature_names_out(columnsToEncode))], axis=1).reindex()
users_pd.head()

max ocupation index:  20
min ocupation index:  0
different zip codes:  3439


/tmp/ipykernel_410703/49124892.py:20: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  users_pd = pd.concat([users_pd.drop(columnsToEncode, 1),
/home/sudakovcom/Desktop/RePlayHDILab2024/.conda/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,user_idx,agegroup_20-29,agegroup_30-39,agegroup_40-49,agegroup_51-60,agegroup_<20,gender_F,gender_M,occupation_0,occupation_1,...,occupation_11,occupation_12,occupation_13,occupation_14,occupation_15,occupation_16,occupation_17,occupation_18,occupation_19,occupation_20
0,4131,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2364,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,4217,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,5916,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1603,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [38]:
#make it pyspark
user_features = spark.createDataFrame(users_pd)
user_features.printSchema()
user_features.show()
print("total users: ",user_features.count())

root
 |-- user_idx: integer (nullable = true)
 |-- agegroup_20-29: double (nullable = true)
 |-- agegroup_30-39: double (nullable = true)
 |-- agegroup_40-49: double (nullable = true)
 |-- agegroup_51-60: double (nullable = true)
 |-- agegroup_<20: double (nullable = true)
 |-- gender_F: double (nullable = true)
 |-- gender_M: double (nullable = true)
 |-- occupation_0: double (nullable = true)
 |-- occupation_1: double (nullable = true)
 |-- occupation_2: double (nullable = true)
 |-- occupation_3: double (nullable = true)
 |-- occupation_4: double (nullable = true)
 |-- occupation_5: double (nullable = true)
 |-- occupation_6: double (nullable = true)
 |-- occupation_7: double (nullable = true)
 |-- occupation_8: double (nullable = true)
 |-- occupation_9: double (nullable = true)
 |-- occupation_10: double (nullable = true)
 |-- occupation_11: double (nullable = true)
 |-- occupation_12: double (nullable = true)
 |-- occupation_13: double (nullable = true)
 |-- occupation_14: double

/home/sudakovcom/Desktop/RePlayHDILab2024/.conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:471: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


## 2.2. Fitting various bandits

In [39]:
bandit_models = {
          'Popular': [PopRec(), 'no_opt'], 
          'Random (uniform)': [RandomRec(seed=SEED, distribution='uniform'), 'no_opt'], 
          'Random (popularity-based)': [RandomRec(seed=SEED, distribution='popular_based'), {"alpha": [-0.5, 100]}],
          'UCB': [UCB(exploration_coef=2.0), 'no_opt'], #2.0 as default, 0.5 as original 
          'Wilson': [Wilson(), 'no_opt'],
          'TS (context-free)': [ThompsonSampling(),'no_opt'],
          'Linear UCB (eps = -10.0)(disjoint)':[LinUCB(eps = -10.0, alpha = 1.0, is_hybrid = False), 'no_opt'],
          'Linear UCB (eps = -10.0)(hybrid)':[LinUCB(eps = -10.0, alpha = 1.0, is_hybrid = True), 'no_opt'],
          'Linear UCB (eps = -5.0)(disjoint)':[LinUCB(eps = -5.0, alpha = 1.0, is_hybrid = False), 'no_opt'],
          'Linear UCB (eps = -5.0)(hybrid)':[LinUCB(eps = -5.0, alpha = 1.0, is_hybrid = True), 'no_opt'],
          'Linear UCB (eps = -2.0)(disjoint)':[LinUCB(eps = -2.0, alpha = 1.0, is_hybrid = False), 'no_opt'],
          'Linear UCB (eps = -2.0)(hybrid)':[LinUCB(eps = -2.0, alpha = 1.0, is_hybrid = True), 'no_opt'],
          'Linear UCB (disjoint models)':[LinUCB(eps = 0.0, alpha = 1.0, is_hybrid = False), {"eps": [-20.0, 10.0], "alpha": [0.001, 10.0]}]
         }

In [40]:
feature_schema = FeatureSchema(
    [
        FeatureInfo(
            column="user_idx",
            feature_type=FeatureType.CATEGORICAL,
            feature_hint=FeatureHint.QUERY_ID,
        ),
        FeatureInfo(
            column="item_idx",
            feature_type=FeatureType.CATEGORICAL,
            feature_hint=FeatureHint.ITEM_ID,
        ),
        FeatureInfo(
            column="relevance",
            feature_type=FeatureType.NUMERICAL,
            feature_hint=FeatureHint.RATING,
        ),
        FeatureInfo(
            column="timestamp",
            feature_type=FeatureType.NUMERICAL,
            feature_hint=FeatureHint.TIMESTAMP,
        ),
    ]
)

In [43]:
all_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=log_replay,
    item_features=item_features,
    query_features=user_features,
    categorical_encoded = True
)

train_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=train,
    item_features=item_features,
    query_features=user_features,
    categorical_encoded = True
)

test_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=test,
    item_features=item_features,
    query_features=user_features,
    categorical_encoded = True
)

train_neg_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=pos_neg_train,
    item_features=item_features,
    query_features=user_features,
    categorical_encoded = True
)

opt_train_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=opt_train,
    item_features=item_features,
    query_features=user_features,
    categorical_encoded = True
)

opt_val_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=opt_val,
    item_features=item_features,
    query_features=user_features,
    categorical_encoded = True
)

In [44]:
e = Experiment(
    [
        MAP(K),
        NDCG(K),
        HitRate(K_list_metrics),
        Coverage(K),
        Surprisal(K),
        MRR(K)
    ],
    test_dataset.interactions,
    train_dataset.interactions,
    query_column=train_dataset.feature_schema.query_id_column,
    item_column=train_dataset.feature_schema.item_id_column,
    rating_column=train_dataset.feature_schema.interactions_rating_column,
    )

In [45]:
import warnings
warnings.filterwarnings("ignore")

In [46]:
%%time
full_pipeline(bandit_models, e, train_dataset, test_dataset)

03-Oct-24 17:33:38, replay, INFO: Popular started
03-Oct-24 17:33:38, replay, INFO: Popular fit_predict started


+--------+--------+-------------------+
|user_idx|item_idx|          relevance|
+--------+--------+-------------------+
|      18|      14| 0.3258278145695364|
|      18|      32| 0.2682119205298013|
|      18|      48|  0.223841059602649|
|      18|      47| 0.2187086092715232|
|      18|      89|0.16258278145695365|
|      18|     114|0.15960264900662252|
|      18|     101| 0.1490066225165563|
|      18|     105| 0.1478476821192053|
|      18|     124|0.14718543046357616|
|      18|     157|0.14089403973509934|
|      46|      12|0.33410596026490064|
|      46|      18| 0.3086092715231788|
|      46|      19| 0.2905629139072848|
|      46|      22| 0.2900662251655629|
|      46|      28| 0.2814569536423841|
|      46|      30| 0.2682119205298013|
|      46|      31|               0.25|
|      46|      35|0.24188741721854304|
|      46|      34|0.23791390728476822|
|      46|      43|0.22996688741721855|
|     186|      12|0.33410596026490064|
|     186|      17| 0.3130794701986755|


03-Oct-24 17:33:53, replay, INFO: Random (uniform) started                      
03-Oct-24 17:33:53, replay, INFO: Random (uniform) fit_predict started


         NDCG@10    MRR@10  Coverage@10  fit_time
Popular  0.24367  0.390414     0.033903  1.755369


+--------+--------+------------------+
|user_idx|item_idx|         relevance|
+--------+--------+------------------+
|      18|     757|0.9993475987324224|
|      18|    1948|0.9958625269029056|
|      18|    1658|0.9958581137949202|
|      18|    1522|0.9951778117229308|
|      18|    3706|0.9930875755613394|
|      18|     197|0.9930827474156511|
|      18|    2644|0.9929310746247295|
|      18|    2730|0.9927206244709196|
|      18|    3202|0.9926568867577302|
|      18|    1782|0.9921495467387857|
|      46|    3827|0.9994992249598874|
|      46|    3207|0.9991521584153258|
|      46|    3222|0.9991089319441182|
|      46|    2128|0.9955462962465224|
|      46|    1540|0.9949174902180918|
|      46|    2273|0.9942066707891665|
|      46|    2286|0.9940611391623346|
|      46|    3396| 0.991560971355034|
|      46|    1596|0.9901570394788072|
|      46|     311|0.9865957485430298|
|     186|    3637|0.9969224342334095|
|     186|     734|0.9964346644226986|
|     186|     493|0.9940

...

+--------+--------+-------------------+
|user_idx|item_idx|          relevance|
+--------+--------+-------------------+
|      18|      32| 0.8405418834740582|
|      18|      14| 0.8316342518691979|
|      18|     146| 0.8013246748718814|
|      18|     114| 0.7946339055381191|
|      18|     215| 0.7897250048553988|
|      18|      47| 0.7896118627912142|
|      18|     378|  0.778693120200573|
|      18|      48| 0.7756432993751232|
|      18|     105| 0.7697081658337579|
|      18|     239| 0.7616835109324535|
|      46|      43| 0.7872416132729293|
|      46|      18| 0.7861638975506179|
|      46|     106| 0.7742611966326691|
|      46|      52| 0.7590022623298114|
|      46|      49| 0.7537616363539821|
|      46|      51| 0.7530864043043335|
|      46|      85| 0.7477091686899544|
|      46|     118| 0.7476233178313462|
|      46|      35| 0.7454942001863504|
|      46|      28| 0.7446695166766942|
|     186|      24| 0.7946514225191219|
|     186|      17| 0.7549099420400205|


03-Oct-24 17:49:37, replay, INFO: Linear UCB (disjoint models) started          
03-Oct-24 17:49:37, replay, INFO: Linear UCB (disjoint models) optimization started
[I 2024-10-03 17:49:37,760] A new study created in memory with name: no-name-a72cf692-0069-43b5-bea9-1b70b1f4b9e7


                                     NDCG@10    MRR@10  Coverage@10   fit_time
Linear UCB (eps = -10.0)(hybrid)    0.256881  0.415585     0.054637  53.074481
Linear UCB (eps = -5.0)(hybrid)     0.256847  0.412047     0.055478  50.796363
Linear UCB (eps = -10.0)(disjoint)  0.256768  0.415579     0.054637  11.314386
Linear UCB (eps = -5.0)(disjoint)   0.256633  0.411666     0.055758  11.281787
Popular                             0.243670  0.390414     0.033903   1.755369
Linear UCB (eps = -2.0)(disjoint)   0.243607  0.389884     0.061362  11.273261
Linear UCB (eps = -2.0)(hybrid)     0.243178  0.388782     0.061082  50.755872
Wilson                              0.092121  0.180976     0.017092   1.060856
Random (popularity-based)           0.026008  0.064604     0.839731   1.767257
Random (uniform)                    0.022416  0.058170     0.942281   1.586289
TS (context-free)                   0.012693  0.027208     0.005884   1.352636
UCB                                 0.000000  0.0000

[I 2024-10-03 17:49:57,680] Trial 0 finished with value: 0.20628550189639538 and parameters: {'eps': -3.0495855167727406, 'alpha': 8.979674297265841}. Best is trial 0 with value: 0.20628550189639538.
[I 2024-10-03 17:50:17,371] Trial 1 finished with value: 0.0034131101044205547 and parameters: {'eps': 0.30618109472649024, 'alpha': 0.7518498633305363}. Best is trial 0 with value: 0.20628550189639538.
[I 2024-10-03 17:50:37,143] Trial 2 finished with value: 0.0008720996683991461 and parameters: {'eps': 4.0811330801808126, 'alpha': 1.7822771570303957}. Best is trial 0 with value: 0.20628550189639538.
[I 2024-10-03 17:50:56,430] Trial 3 finished with value: 0.20571251265206217 and parameters: {'eps': -10.773006536288307, 'alpha': 4.620065101743285}. Best is trial 0 with value: 0.20628550189639538.
[I 2024-10-03 17:51:15,587] Trial 4 finished with value: 0.20623228824713366 and parameters: {'eps': -4.806488909745973, 'alpha': 0.865203088560681}. Best is trial 0 with value: 0.206285501896395

+--------+--------+-------------------+
|user_idx|item_idx|          relevance|
+--------+--------+-------------------+
|      18|      14| 0.7694890117395012|
|      18|      32| 0.7671931519224076|
|      18|     146| 0.7102568089297239|
|      18|     114| 0.7091622213419695|
|      18|      47|  0.708968114382001|
|      18|      48| 0.6955521929385429|
|      18|     215| 0.6866091981790141|
|      18|     105| 0.6705292239333451|
|      18|     378| 0.6572079886344135|
|      18|      89| 0.6506418011200548|
|      46|      18|  0.698125042723037|
|      46|      43| 0.6826614210683251|
|      46|     106| 0.6563958014803232|
|      46|      52| 0.6553195264830027|
|      46|      28| 0.6428811803345469|
|      46|      49| 0.6425797963195734|
|      46|      51| 0.6389495000741722|
|      46|      85|  0.638089585001715|
|      46|      22| 0.6378944580559095|
|      46|      35| 0.6342055341709173|
|     186|      24| 0.6943580745900739|
|     186|      17| 0.6518178242235573|


In [51]:
e.results.sort_values('NDCG@10', ascending=False)

,MAP@10,NDCG@10,HitRate@1,HitRate@5,HitRate@10,Coverage@10,Surprisal@10,MRR@10,fit_time,predict_time,metric_time,full_time,params
Linear UCB (eps = -10.0)(hybrid),0.167964,0.256881,0.311677,0.561896,0.659350,0.054637,0.125022,0.415585,53.074481,199.519035,7.204302,259.797818,"{'regression type': 'hybrid', 'seed': None}"
Linear UCB (eps = -5.0)(hybrid),0.168116,0.256847,0.308165,0.568042,0.655838,0.055478,0.126488,0.412047,50.796363,199.043191,7.027678,256.867232,"{'regression type': 'hybrid', 'seed': None}"
Linear UCB (eps = -10.0)(disjoint),0.167851,0.256768,0.311677,0.561896,0.659350,0.054637,0.125029,0.415579,11.314386,5.456721,5.704894,22.476000,"{'regression type': 'disjoint', 'seed': None}"
Linear UCB (eps = -5.0)(disjoint),0.167926,0.256633,0.307287,0.567164,0.656716,0.055758,0.126491,0.411666,11.281787,5.287628,5.680037,22.249452,"{'regression type': 'disjoint', 'seed': None}"
Linear UCB (disjoint models),0.162927,0.251921,0.293240,0.561018,0.657594,0.055758,0.129758,0.402974,11.375094,5.489639,5.716758,22.581491,"{'regression type': 'disjoint', 'seed': None}"
Popular,0.157257,0.243670,0.284460,0.530290,0.645303,0.033903,0.118354,0.390414,1.755369,5.097414,8.111187,14.963971,"{'use_rating': False, 'add_cold_items': True, ..."
Linear UCB (eps = -2.0)(disjoint),0.155757,0.243607,0.271291,0.558385,0.654083,0.061362,0.134384,0.389884,11.273261,5.326424,5.731244,22.330929,"{'regression type': 'disjoint', 'seed': None}"
Linear UCB (eps = -2.0)(hybrid),0.155702,0.243178,0.269535,0.558385,0.653205,0.061082,0.134378,0.388782,50.755872,199.113863,7.220896,257.090631,"{'regression type': 'hybrid', 'seed': None}"
Wilson,0.045002,0.092121,0.083406,0.345040,0.414399,0.017092,0.262190,0.180976,1.060856,4.335059,6.024863,11.420777,"{'alpha': 0.05, 'add_cold_items': True, 'cold_..."
Random (popularity-based),0.009357,0.026008,0.028973,0.110623,0.186128,0.839731,0.501055,0.064604,1.767257,4.451728,5.792844,12.011829,"{'distribution': 'popular_based', 'alpha': 89...."
